<a href="https://colab.research.google.com/github/BlackHat-17/IEEE-Research-Project-24-25/blob/dev/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numba import cuda,float32
import math

In [ ]:

@cuda.jit
def f(a,b,c):
    pos=cuda.grid(1)
    size=len(c)
    if pos<size :
        c[pos]=a[pos]+b[pos]

In [ ]:
N=10000
a=cuda.to_device(np.random.random(N))
b=cuda.to_device(np.random.random(N))
c=cuda.device_array_like(a)
Tpb=16

In [ ]:
nthreads=256
nblocks=(len(a)//nthreads)+1
f[nblocks,nthreads](a,b,c)
c.copy_to_host()

/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 40 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


array([0.77303233, 1.03776897, 1.60943586, ..., 1.02424524, 0.83423695,
       1.32145656])

In [ ]:
@cuda.jit
def matrix_multiply(matrix1,matrix2,res_matrix):
    A=cuda.shared.array(shape=(Tpb,Tpb), dtype=float32)
    B=cuda.shared.array(shape=(Tpb,Tpb), dtype=float32)
    x,y=cuda.grid(2)
    tx=cuda.threadIdx.x
    ty=cuda.threadIdx.y
    bpg=cuda.blockDim.x
    temp=float32(0.)
    for k in range(bpg):
        temp+=matrix1[x,k]*matrix2[k,y]
        res_matrix[x,y]=temp

In [ ]:

print("matrix 1")
m1=int(input("Enter no. of rows:"))
n1=int(input("Enter no. of columns:"))
x_h=np.arange(m1*n1).reshape([m1, n1])
print(x_h)

matrix 1
Enter no. of rows:3
Enter no. of columns:3


In [ ]:
print("matrix 2")
m2=int(input("Enter no. of rows:"))
n2=int(input("Enter no. of columns:"))
y_h=np.arange(m2*n2).reshape([m2, n2])
print(y_h)

matrix 2
Enter no. of rows:3
Enter no. of columns:3
[[0 1 2]
 [3 4 5]
 [6 7 8]]


In [ ]:
z_h=np.zeros([m1,n2])
x_d=cuda.to_device(x_h)
y_d=cuda.to_device(y_h)
z_d=cuda.to_device(z_h)

In [ ]:

threadsperblock=(16, 16)
blockspergrid_x= math.ceil(z_h.shape[0]/threadsperblock[0])
blockspergrid_y= math.ceil(z_h.shape[1]/threadsperblock [1])
blockspergrid=(blockspergrid_x, blockspergrid_y)

In [ ]:
matrix_multiply[blockspergrid,threadsperblock](x_d,y_d,z_d)

/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [ ]:
z_h=z_d.copy_to_host()
print(z_h)

[[15. 18. 21.]
 [ 6.  7. 66.]
 [ 0.  0.  0.]]
